In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
import torch
import pickle
import math
import prior.CustomDistribution as custom
import config
import time
import IPython.display as IPd
import warnings

from forward import Forward 
from survey import Survey

from sbi import utils as utils
from sbi.utils import user_input_checks as uic

from util import *
from block_utils import *
from pygimli_utils import *
from polynomials import *

from sklearn.metrics import mean_squared_error


_ = torch.manual_seed(0)

2024-08-06 16:36:40.284907: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 16:36:40.324865: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 16:36:40.324907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 16:36:40.324930: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 16:36:40.333315: I tensorflow/core/platform/cpu_feature_g

In [2]:
max_depth= 23.5
step_size= 0.5
num_points= int(max_depth/step_size) +1 
max_resistivity = 1000
min_resistivity = 1
num_measurements = 23
num_dim = 48

depths = np.linspace(0, max_depth, num=num_dim)
thicknesses = torch.ones(num_dim) * 0.5

## Load all posterior models

In [3]:
# **** Step prior
filename_step_model = "pickled_objects/steps_100k_1000_ohm_48points_new.pkl"
inference_step, posterior_step = unpickle_objects(filename_step_model)


# **** Polynomial Prior
filename_legendre_model = "pickled_objects/legendre_100k_1000_ohm_48points_new.pkl"
inference_poly, posterior_poly = unpickle_objects(filename_legendre_model)

# **** Independent Step Prior
filename_ind_steps_model = "pickled_objects/ind_steps_500k_1000_ohm_48points.pkl"
inference_ind_step, posterior_ind_step = unpickle_objects(filename_ind_steps_model)

# **** Block Prior 
filename_block_model = "pickled_objects/block_100k_1000_ohm_48points.pkl"
inference_block, posterior_block = unpickle_objects(filename_block_model)

## Load the prior samples that were generated in the Synthetic Test Case Notebook

In [4]:
filename_step = 'data/prior_samples/step_samples.pkl'
filename_poly = 'data/prior_samples/poly_samples.pkl'
filename_block = 'data/prior_samples/block_samples.pkl'
filename_ind_step = 'data/prior_samples/ind_step_samples.pkl'


(app_res_samples_step, prior_samples_step) = unpickle_objects(filename_step)
(app_res_samples_poly, prior_samples_poly) = unpickle_objects(filename_poly)
(app_res_samples_block, prior_samples_block)= unpickle_objects(filename_block)
(app_res_samples_ind_step, prior_samples_ind_step) = unpickle_objects(filename_ind_step)

## Pygimli Inversion

In [5]:
pygimili_inv_filename_steps = 'data/inversion/prior_samples/step-samples-{}-layer.pkl'
pygimili_inv_filename_polys = 'data/inversion/prior_samples/poly-samples-{}-layer.pkl'
pygimili_inv_filename_blocks = 'data/inversion/prior_samples/block-samples-{}-layer.pkl'
pygimili_inv_filename_ind_steps = 'data/inversion/prior_samples/ind-step-samples-{}-layer.pkl'


inv_results_step_4 = unpickle_objects(pygimili_inv_filename_steps.format(4))
inv_results_step_5 = unpickle_objects(pygimili_inv_filename_steps.format(5))
inv_results_step_48 = unpickle_objects(pygimili_inv_filename_steps.format(48))


inv_results_poly_4 = unpickle_objects(pygimili_inv_filename_polys.format(4))
inv_results_poly_5 = unpickle_objects(pygimili_inv_filename_polys.format(5))
inv_results_poly_48 = unpickle_objects(pygimili_inv_filename_polys.format(48))



inv_results_block_4 = unpickle_objects(pygimili_inv_filename_blocks.format(4))
inv_results_block_5 = unpickle_objects(pygimili_inv_filename_blocks.format(5))
inv_results_block_48 = unpickle_objects(pygimili_inv_filename_blocks.format(48))



inv_results_ind_step_4 = unpickle_objects(pygimili_inv_filename_ind_steps.format(4))
inv_results_ind_step_5 = unpickle_objects(pygimili_inv_filename_ind_steps.format(5))
inv_results_ind_step_48 = unpickle_objects(pygimili_inv_filename_ind_steps.format(48))


#### Evaluation of step sample inversion

In [6]:
inv_step_models_4 = [r[0] for r in inv_results_step_4]
inv_step_inv_response_4 = [r[1] for r in inv_results_step_4]
processed_resistivity_inv_step_4 = [process_gimli_results(model) for model in inv_step_models_4]


rmses_param_pygim_step_4  = [rmse(step_sample, res[:48]) for step_sample, res in zip(prior_samples_step, processed_resistivity_inv_step_4) ]
print('Avg RMSE parameter space 4 layers:', np.average(rmses_param_pygim_step_4))
rmses_obs_pygim_sep_4 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_step, inv_step_inv_response_4)] 
print('Avg RMSE observation space 4 layers:', np.average(rmses_obs_pygim_sep_4))

Avg RMSE parameter space 4 layers: 104.81828
Avg RMSE observation space 4 layers: 5.7715082794419965


In [7]:
inv_step_models_5 = [r[0] for r in inv_results_step_5]
inv_step_inv_response_5 = [r[1] for r in inv_results_step_5]
processed_resistivity_inv_step_5 = [process_gimli_results(model) for model in inv_step_models_5]


rmses_param_pygim_step_5  = [rmse(step_sample, res[:48]) for step_sample, res in zip(prior_samples_step, processed_resistivity_inv_step_5) ]
print('Avg RMSE parameter space 5 layers:', np.average(rmses_param_pygim_step_5))
rmses_obs_pygim_sep_5 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_step, inv_step_inv_response_5)] 
print('Avg RMSE observation space 5 layers:', np.average(rmses_obs_pygim_sep_5))

Avg RMSE parameter space 5 layers: 101.95918
Avg RMSE observation space 5 layers: 3.759240397491728


In [8]:
inv_step_models_48 = [r[0] for r in inv_results_step_48]
inv_step_inv_response_48 = [r[1] for r in inv_results_step_48]
processed_resistivity_inv_step_48 = [process_gimli_results(model) for model in inv_step_models_48]

rmses_param_pygim_step_48  = [rmse(step_sample, res[:48]) for step_sample, res in zip(prior_samples_step, processed_resistivity_inv_step_48) ]
print('Avg RMSE parameter space 48 layers:', np.average(rmses_param_pygim_step_48))
rmses_obs_pygim_sep_48 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_step, inv_step_inv_response_48)] 
print('Avg RMSE observation space 48 layers:', np.average(rmses_obs_pygim_sep_48))

Avg RMSE parameter space 48 layers: 84.881065
Avg RMSE observation space 48 layers: 3.9289012174240114


#### Evaluation of Poly sample inversion

In [9]:
polynomial = Polynomial(max_depth=max_depth, num_dim=num_dim, max_res=max_resistivity, num_measurements=num_measurements)

In [10]:
inv_poly_models_4 = [r[0] for r in inv_results_poly_4]
inv_poly_inv_response_4 = [r[1] for r in inv_results_poly_4]
processed_resistivity_inv_poly_4 = [process_gimli_results(model) for model in inv_poly_models_4]

rmses_param_pygim_poly_4  = [rmse(polynomial.coefficients_to_resistivity(poly_sample), res[:48]) for poly_sample, res in zip(prior_samples_poly, processed_resistivity_inv_poly_4) ]
print('Avg RMSE parameter space 4 layers:', np.average(rmses_param_pygim_poly_4))
rmses_obs_pygim_poly_4 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_poly, inv_poly_inv_response_4)] 
print('Avg RMSE observation space 4 layers:', np.average(rmses_obs_pygim_poly_4))

Avg RMSE parameter space 4 layers: 103.15662906802763
Avg RMSE observation space 4 layers: 5.219796773822577


In [11]:
inv_poly_models_5 = [r[0] for r in inv_results_poly_5]
inv_poly_inv_response_5 = [r[1] for r in inv_results_poly_5]
processed_resistivity_inv_poly_5 = [process_gimli_results(model) for model in inv_poly_models_5]


rmses_param_pygim_poly_5  = [rmse(polynomial.coefficients_to_resistivity(poly_sample), res[:48]) for poly_sample, res in zip(prior_samples_poly, processed_resistivity_inv_poly_5) ]
print('Avg RMSE parameter space 5 layers:', np.average(rmses_param_pygim_poly_5))
rmses_obs_pygim_poly_5 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_poly, inv_poly_inv_response_5)] 
print('Avg RMSE observation space 5 layers:', np.average(rmses_obs_pygim_poly_5))

Avg RMSE parameter space 5 layers: 118.21289992993422
Avg RMSE observation space 5 layers: 3.339443419346934


In [12]:
inv_poly_models_48 = [r[0] for r in inv_results_poly_48]
inv_poly_inv_response_48 = [r[1] for r in inv_results_poly_48]
processed_resistivity_inv_poly_48 = [process_gimli_results(model) for model in inv_poly_models_48]


rmses_param_pygim_poly_48  = [rmse(polynomial.coefficients_to_resistivity(poly_sample), res[:48]) for poly_sample, res in zip(prior_samples_poly, processed_resistivity_inv_poly_48) ]
print('Avg RMSE parameter space 48 layers:', np.average(rmses_param_pygim_poly_48))
rmses_obs_pygim_poly_48 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_poly, inv_poly_inv_response_48)] 
print('Avg RMSE observation space 48 layers:', np.average(rmses_obs_pygim_poly_48))

Avg RMSE parameter space 48 layers: 113.50087488541362
Avg RMSE observation space 48 layers: 4.469134235273012


#### Evaluation of Block sample inversion

In [13]:
def ensure_48_points(block_res):
    '''    
    Check if the depth profile has 48 values. If depth profile is too short, extend the last layers -> used for plotting
    in case the model has 48 points it is not modified. If it has less, the resistivity values of the last layer are appenden to ensure 48 points.
    
    Return block resistivities
    '''
    
    # Check if the array has less than 48 values
    while len(block_res) < 48:
        # Append the last value to the array until it has 48 values
        block_res.append(block_res[-1])
    return np.array(block_res)
    


def block_sample_to_resistivity(sample): 
    '''
    Transforms a block prior or posterior sample into the resistivity depth profile.
    The sample is splitted into layer thicknesses and resistivity values. 
    From this a depth profile of at least 48 steps is generated that representens the resistivities at small layer steps of 0.5m 
    to compare to other depth profiles of other priors or posteriors
    
    Returns the resistivity depth profile from a block sample. 
    '''
    
    inv_res_thick_depth, max_depth = transfrom_and_find_max_reached_depth([sample])
    mapped_sample = map_resistivities_to_depth(inv_res_thick_depth, max_depth)
    low_res_sample = transform_to_lower_res_samples(mapped_sample)[0]
    return ensure_48_points(low_res_sample)


In [14]:
inv_block_models_4 = [r[0] for r in inv_results_block_4]
inv_block_inv_response_4 = [r[1] for r in inv_results_block_4]
processed_resistivity_inv_block_4 = [process_gimli_results(model) for model in inv_block_models_4]


rmses_param_pygim_block_4  = [rmse(block_sample_to_resistivity(block_sample), res[:48]) for block_sample, res in zip(prior_samples_block, processed_resistivity_inv_block_4) ]
print('Avg RMSE parameter space 4 layers:', np.average(rmses_param_pygim_block_4))
rmses_obs_pygim_block_4 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_block, inv_block_inv_response_4)] 
print('Avg RMSE observation space 4 layers:', np.average(rmses_obs_pygim_block_4))

Avg RMSE parameter space 4 layers: 93.53353061009314
Avg RMSE observation space 4 layers: 4.563220472856642


In [15]:
inv_block_models_5 = [r[0] for r in inv_results_block_5]
inv_block_inv_response_5 = [r[1] for r in inv_results_block_5]
processed_resistivity_inv_block_5 = [process_gimli_results(model) for model in inv_block_models_5]


rmses_param_pygim_block_5  = [rmse(block_sample_to_resistivity(block_sample), res[:48]) for block_sample, res in zip(prior_samples_block, processed_resistivity_inv_block_5) ]
print('Avg RMSE parameter space 5 layers:', np.average(rmses_param_pygim_block_5))
rmses_obs_pygim_block_5 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_block, inv_block_inv_response_5)] 
print('Avg RMSE observation space 5 layers:', np.average(rmses_obs_pygim_block_5))

Avg RMSE parameter space 5 layers: 139.69246167362616
Avg RMSE observation space 5 layers: 3.7821560277384476


In [16]:
inv_block_models_48 = [r[0] for r in inv_results_block_48]
inv_block_inv_response_48 = [r[1] for r in inv_results_block_48]
processed_resistivity_inv_block_48 = [process_gimli_results(model) for model in inv_block_models_48]


rmses_param_pygim_block_48  = [rmse(block_sample_to_resistivity(block_sample), res[:48]) for block_sample, res in zip(prior_samples_block, processed_resistivity_inv_block_48) ]
print('Avg RMSE parameter space 48 layers:', np.average(rmses_param_pygim_block_48))
rmses_obs_pygim_block_48 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_block, inv_block_inv_response_48)] 
print('Avg RMSE observation space 48 layers:', np.average(rmses_obs_pygim_block_48))

Avg RMSE parameter space 48 layers: 118.74335785447855
Avg RMSE observation space 48 layers: 4.309729593188939


#### Evaluation of Ind Step sample inversion

In [17]:
inv_ind_step_models_4 = [r[0] for r in inv_results_ind_step_4]
inv_ind_step_inv_response_4 = [r[1] for r in inv_results_ind_step_4]
processed_resistivity_inv_ind_step_4 = [process_gimli_results(model) for model in inv_ind_step_models_4]


rmses_param_pygim_ind_step_4  = [rmse(ind_step_sample, res[:48]) for ind_step_sample, res in zip(prior_samples_ind_step, processed_resistivity_inv_ind_step_4) ]
print('Avg RMSE parameter space 4 layers:', np.average(rmses_param_pygim_ind_step_4))
rmses_obs_pygim_sep_4 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_ind_step, inv_ind_step_inv_response_4)] 
print('Avg RMSE observation space 4 layers:', np.average(rmses_obs_pygim_sep_4))

Avg RMSE parameter space 4 layers: 320.49292
Avg RMSE observation space 4 layers: 6.798597550498842


In [18]:
inv_ind_step_models_5 = [r[0] for r in inv_results_ind_step_5]
inv_ind_step_inv_response_5 = [r[1] for r in inv_results_ind_step_5]
processed_resistivity_inv_ind_step_5 = [process_gimli_results(model) for model in inv_ind_step_models_5]


rmses_param_pygim_ind_step_5  = [rmse(ind_step_sample, res[:48]) for ind_step_sample, res in zip(prior_samples_ind_step, processed_resistivity_inv_ind_step_5) ]
print('Avg RMSE parameter space 5 layers:', np.average(rmses_param_pygim_ind_step_5))
rmses_obs_pygim_sep_5 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_ind_step, inv_ind_step_inv_response_5)] 
print('Avg RMSE observation space 5 layers:', np.average(rmses_obs_pygim_sep_5))

Avg RMSE parameter space 5 layers: 324.7596
Avg RMSE observation space 5 layers: 4.852102298775273


In [19]:
inv_ind_step_models_48 = [r[0] for r in inv_results_ind_step_48]
inv_ind_step_inv_response_48 = [r[1] for r in inv_results_ind_step_48]
processed_resistivity_inv_ind_step_48 = [process_gimli_results(model) for model in inv_ind_step_models_48]


rmses_param_pygim_ind_step_48  = [rmse(ind_step_sample, res[:48]) for ind_step_sample, res in zip(prior_samples_ind_step, processed_resistivity_inv_ind_step_48) ]
print('Avg RMSE parameter space 48 layers:', np.average(rmses_param_pygim_ind_step_48))
rmses_obs_pygim_sep_48 = [rmse(app_res_prior, inv_response)   for app_res_prior, inv_response in zip(app_res_samples_ind_step, inv_ind_step_inv_response_48)] 
print('Avg RMSE observation space 48 layers:', np.average(rmses_obs_pygim_sep_48))

Avg RMSE parameter space 48 layers: 331.42914
Avg RMSE observation space 48 layers: 4.9405356420362825
